In [13]:
# %load process.py
import numpy as np
import pandas as pd
import tushare as ts

class process_stock():

    def __init__(self, stock_list, token):
        self.stock_list = stock_list
        self.token = token
        ts.set_token(token)
        self.pro = ts.pro_api()
        
    def get_stock(self, name):
        '''
        日交易额单位：千元*N_amount
        '''
        df = self.pro.daily(ts_code = name, start_date = self.start_date, end_date=self.end_date)
        df['yield_rate'] = df['change']/df['pre_close']
        df['abs_yield_rate'] = abs(np.log(df['close']/df['pre_close']))
        df['amihud'] = df['abs_yield_rate']/(df['amount']/self.N_amount)
        return df
    

    
    def process_stock(self, name, df):
        '''
        '''
        df = df[['trade_date','yield_rate','amihud']]
        df.rename(columns={'yield_rate':'yield_rate_'+name, 'amihud':'amihud_'+name},inplace=True)
        return df

    def create_time_series(self, start_date, end_date, N_amount=1):
        '''
        获取 stock_list 中每只 stock 的 yield rate 以及 amihud 指标并得到时间序列
        
        Args:
            name: stock name
            start_date: trade start date
            end_date: trade end date
            N_amount: 成交额的单位
        '''
        # 先获取第一个
        self.start_date = start_date
        self.end_date = end_date
        self.N_amount = N_amount
        df = self.get_stock(self.stock_list[0])
        df = self.process_stock(self.stock_list[0], df)
        # 将第一个和后面的合并
        for i in range(len(self.stock_list)-1):
            df_temp = self.get_stock(self.stock_list[i+1])
            df_temp = self.process_stock(stock_list[i+1], df_temp)
            df = pd.merge(df,df_temp)
        self.df = df
        # 改变顺序
        row_list1 = ['trade_date']
        row_list2 = []
        for name in self.stock_list:
            row_list1.append('yield_rate_'+name)
            row_list2.append('amihud_'+name)
        self.df = self.df[row_list1+row_list2]
        return self.df

    def save_df(self, path):
        self.df.to_csv(path + 'stock.csv', index = False)
        


In [14]:
stock_list = ['000001.SZ','600000.SH','600015.SH','600016.SH','600036.SH','601166.SH',
              '601288.SH','601328.SH','601398.SH','601818.SH',
              '601939.SH','601988.SH','601998.SH']
token = '094f15d71394516b730602faa77b1c708007b8d05df300590b4445ed'
process = process_stock(stock_list, token)

In [15]:
df = process.create_time_series('20190101','20191231', 1000)
df

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4301: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,trade_date,yield_rate_000001.SZ,yield_rate_600000.SH,yield_rate_600015.SH,yield_rate_600016.SH,yield_rate_600036.SH,yield_rate_601166.SH,yield_rate_601288.SH,yield_rate_601328.SH,yield_rate_601398.SH,...,amihud_600016.SH,amihud_600036.SH,amihud_601166.SH,amihud_601288.SH,amihud_601328.SH,amihud_601398.SH,amihud_601818.SH,amihud_601939.SH,amihud_601988.SH,amihud_601998.SH
0,20191231,-0.007242,0.002431,0.002614,0.000000,-0.006609,0.002024,-0.008065,-0.001773,-0.001698,...,0.000000,4.692472e-06,0.000002,0.000011,0.000005,0.000002,0.000013,0.000000,0.000020,0.000000
1,20191230,-0.003608,0.001623,0.002621,0.006380,-0.000792,-0.010020,-0.002681,0.003559,-0.001695,...,0.000016,4.746218e-07,0.000004,0.000003,0.000007,0.000001,0.000020,0.000003,0.000011,0.000010
2,20191227,0.009715,0.002441,0.005270,0.003200,-0.006299,0.000000,0.005391,0.003571,0.006826,...,0.000007,4.611380e-06,0.000000,0.000007,0.000007,0.000009,0.000006,0.000005,0.000008,0.000025
3,20191226,0.010429,0.004085,0.005298,0.003210,0.008203,0.010121,0.010899,0.005386,0.005146,...,0.000012,1.122201e-05,0.000015,0.000023,0.000019,0.000010,0.000047,0.000024,0.000025,0.000113
4,20191225,-0.006098,-0.003257,-0.005270,-0.004792,-0.005788,-0.007035,-0.002717,-0.001792,-0.005119,...,0.000023,6.526291e-06,0.000009,0.000008,0.000008,0.000011,0.000012,0.000009,0.000012,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,20190108,-0.008214,-0.002004,-0.005369,-0.001736,-0.011755,0.003955,-0.011204,0.003431,-0.007634,...,0.000007,1.738262e-05,0.000011,0.000016,0.000015,0.000013,0.000033,0.000000,0.000020,0.000024
240,20190107,-0.001026,0.002008,-0.005340,-0.003460,0.000392,-0.001974,-0.005571,-0.005119,-0.005693,...,0.000009,3.371224e-07,0.000004,0.000007,0.000018,0.000011,0.000000,0.000008,0.000007,0.000042
241,20190104,0.050647,0.015291,0.019048,0.019400,0.025322,0.023569,0.014124,0.013841,0.013462,...,0.000040,1.479300e-05,0.000033,0.000019,0.000031,0.000021,0.000069,0.000035,0.000019,0.000167
242,20190103,0.009793,0.011340,0.005472,0.003540,0.012617,0.007463,0.005682,0.019400,0.000000,...,0.000010,1.349580e-05,0.000019,0.000011,0.000052,0.000000,0.000105,0.000007,0.000008,0.000026


In [16]:
ts.set_token('094f15d71394516b730602faa77b1c708007b8d05df300590b4445ed')
pro = ts.pro_api()

df_1 = pro.daily(ts_code='601658.SH', start_date='20190101', end_date='20191231')
df_1

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,601658.SH,20191231,5.80,5.88,5.79,5.86,5.81,0.05,0.8606,1224430.80,715889.070
1,601658.SH,20191230,5.71,5.91,5.66,5.81,5.74,0.07,1.2195,1789527.54,1039987.974
2,601658.SH,20191227,5.68,5.76,5.67,5.74,5.69,0.05,0.8787,1309059.32,749825.057
3,601658.SH,20191226,5.68,5.69,5.63,5.69,5.69,0.00,0.0000,1185756.93,670892.421
4,601658.SH,20191225,5.78,5.80,5.68,5.69,5.80,-0.11,-1.8966,1568358.20,898134.028
5,601658.SH,20191224,5.83,5.83,5.74,5.80,5.82,-0.02,-0.3436,1311897.34,758582.939
6,601658.SH,20191223,5.76,5.88,5.74,5.82,5.75,0.07,1.2174,1709986.38,994892.791
7,601658.SH,20191220,5.75,5.84,5.72,5.75,5.74,0.01,0.1742,1762430.52,1017976.197
8,601658.SH,20191219,5.73,5.83,5.70,5.74,5.72,0.02,0.3497,1410280.76,811746.305
9,601658.SH,20191218,5.78,5.84,5.71,5.72,5.85,-0.13,-2.2222,2200642.05,1268629.909


In [ ]:
df_2 = pro.daily(ts_code='601658.SH', start_date='20190101', end_date='20191231')